# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f29809d3190>
├── 'a' --> tensor([[ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923]])
└── 'x' --> <FastTreeValue 0x7f2980a38dc0>
    └── 'c' --> tensor([[ 0.5131,  0.3777, -1.3274, -0.5428],
                        [-2.0245,  0.4623,  0.0510,  0.2346],
                        [ 1.1759,  1.1160,  0.5754,  0.3365]])

In [4]:
t.a

tensor([[ 0.4394,  0.1628,  0.8419],
        [-0.6039, -1.8460, -0.1923]])

In [5]:
%timeit t.a

65.2 ns ± 0.0168 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f29809d3190>
├── 'a' --> tensor([[ 0.9763, -0.1104, -0.9499],
│                   [-1.8339,  2.2687, -1.1935]])
└── 'x' --> <FastTreeValue 0x7f2980a38dc0>
    └── 'c' --> tensor([[ 0.5131,  0.3777, -1.3274, -0.5428],
                        [-2.0245,  0.4623,  0.0510,  0.2346],
                        [ 1.1759,  1.1160,  0.5754,  0.3365]])

In [7]:
%timeit t.a = new_value

66.4 ns ± 0.0213 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4394,  0.1628,  0.8419],
               [-0.6039, -1.8460, -0.1923]]),
    x: Batch(
           c: tensor([[ 0.5131,  0.3777, -1.3274, -0.5428],
                      [-2.0245,  0.4623,  0.0510,  0.2346],
                      [ 1.1759,  1.1160,  0.5754,  0.3365]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4394,  0.1628,  0.8419],
        [-0.6039, -1.8460, -0.1923]])

In [11]:
%timeit b.a

54.1 ns ± 0.068 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1948,  0.1656,  0.5722],
               [ 0.7992, -0.5875, -0.0680]]),
    x: Batch(
           c: tensor([[ 0.5131,  0.3777, -1.3274, -0.5428],
                      [-2.0245,  0.4623,  0.0510,  0.2346],
                      [ 1.1759,  1.1160,  0.5754,  0.3365]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.243 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

856 ns ± 0.129 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 48.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 417 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 415 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f28d56e8c10>
├── 'a' --> tensor([[[ 0.4394,  0.1628,  0.8419],
│                    [-0.6039, -1.8460, -0.1923]],
│           
│                   [[ 0.4394,  0.1628,  0.8419],
│                    [-0.6039, -1.8460, -0.1923]],
│           
│                   [[ 0.4394,  0.1628,  0.8419],
│                    [-0.6039, -1.8460, -0.1923]],
│           
│                   [[ 0.4394,  0.1628,  0.8419],
│                    [-0.6039, -1.8460, -0.1923]],
│           
│                   [[ 0.4394,  0.1628,  0.8419],
│                    [-0.6039, -1.8460, -0.1923]],
│           
│                   [[ 0.4394,  0.1628,  0.8419],
│                    [-0.6039, -1.8460, -0.1923]],
│           
│                   [[ 0.4394,  0.1628,  0.8419],
│                    [-0.6039, -1.8460, -0.1923]],
│           
│                   [[ 0.4394,  0.1628,  0.8419],
│                    [-0.6039, -1.8460, -0.1923]]])
└── 'x' --> <FastTreeValue 0x7f28d56d5400>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f28d5d8dcd0>
├── 'a' --> tensor([[ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923],
│                   [ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923],
│                   [ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923],
│                   [ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923],
│                   [ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923],
│                   [ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923],
│                   [ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923],
│                   [ 0.4394,  0.1628,  0.8419],
│                   [-0.6039, -1.8460, -0.1923]])
└── 'x' --> <FastTreeValue 0x7f29809e0c10>
    └── 'c' --> tensor([[ 0.5131,  0.3777, -1.3274, -0.5428],
                        [-2.0245,  0.4623,  0.0510,  0.2346],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 44.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 54.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5131,  0.3777, -1.3274, -0.5428],
                       [-2.0245,  0.4623,  0.0510,  0.2346],
                       [ 1.1759,  1.1160,  0.5754,  0.3365]],
              
                      [[ 0.5131,  0.3777, -1.3274, -0.5428],
                       [-2.0245,  0.4623,  0.0510,  0.2346],
                       [ 1.1759,  1.1160,  0.5754,  0.3365]],
              
                      [[ 0.5131,  0.3777, -1.3274, -0.5428],
                       [-2.0245,  0.4623,  0.0510,  0.2346],
                       [ 1.1759,  1.1160,  0.5754,  0.3365]],
              
                      [[ 0.5131,  0.3777, -1.3274, -0.5428],
                       [-2.0245,  0.4623,  0.0510,  0.2346],
                       [ 1.1759,  1.1160,  0.5754,  0.3365]],
              
                      [[ 0.5131,  0.3777, -1.3274, -0.5428],
                       [-2.0245,  0.4623,  0.0510,  0.2346],
                       [ 1.1759,  1.1160,  0.5754,  0.3365]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5131,  0.3777, -1.3274, -0.5428],
                      [-2.0245,  0.4623,  0.0510,  0.2346],
                      [ 1.1759,  1.1160,  0.5754,  0.3365],
                      [ 0.5131,  0.3777, -1.3274, -0.5428],
                      [-2.0245,  0.4623,  0.0510,  0.2346],
                      [ 1.1759,  1.1160,  0.5754,  0.3365],
                      [ 0.5131,  0.3777, -1.3274, -0.5428],
                      [-2.0245,  0.4623,  0.0510,  0.2346],
                      [ 1.1759,  1.1160,  0.5754,  0.3365],
                      [ 0.5131,  0.3777, -1.3274, -0.5428],
                      [-2.0245,  0.4623,  0.0510,  0.2346],
                      [ 1.1759,  1.1160,  0.5754,  0.3365],
                      [ 0.5131,  0.3777, -1.3274, -0.5428],
                      [-2.0245,  0.4623,  0.0510,  0.2346],
                      [ 1.1759,  1.1160,  0.5754,  0.3365],
                      [ 0.5131,  0.3777, -1.3274, -0.5428],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 867 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
